# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Answer all statements addressed below as specified in the instructions for the project, covering all necessary details. Please be clear and concise in your answers. Each response should be at most 3 sentences. Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning
You are required to provide data collection and cleaning for the three (3) minimum datasets. Create a function for each of the following sections that reads or scrapes data from a file or website, manipulate and cleans the parsed data, and writes the cleaned data into a new file. 

Make sure your data cleaning and manipulation process is not too simple. Performing complex manipulation and using modules not taught in class shows effort, which will increase the chance of receiving full credit.


## Data Sources
Include sources (as links) to your datasets. Add any additional data sources if needed. Clearly indicate if a data source is different from one submitted in your Phase I, as we will check that it satisfies the requirements.
*   Downloaded Dataset Source: https://www.cdc.gov/brfss/annual_data/2022/files/LLCP2022ASC.zip
*   Web Collection #1 Source: https://www.cdc.gov/brfss/annual_data/2022/zip/codebook22_llcp-v2-508.zip
*   Web Collection #2 Source: https://data.cdc.gov/resource/hksd-2xuw.json



## Downloaded Dataset Requirement

Fill in the predefined functions with your data scraping/parsing code. You may modify/rename each function as you seem fit, but you must provide at least 3 separate functions that clean each of your required datasets.


In [1]:
import pandas as pd
import numpy as np

def data_parser():
  colspecs = [(113, 115), (1900, 1901), (1997, 2001), (185, 187), (168, 169)]
  names = ["SLEPTIM1", "_MENT14D", "_BMI5", "INCOME3", "EDUCA"]
  
  df = pd.read_fwf("LLCP2022.txt", colspecs=colspecs, names=names)
  
  invalid_codes = {
        "SLEPTIM1": [77, 99],
        "_MENT14D": [9],
        "INCOME3": [77, 99],
        "EDUCA": [9]
    }
    
  for col, codes in invalid_codes.items():
      df[col] = df[col].replace(codes, np.nan)
  
  df = df.astype(float)
  
  df = df[
      ((df["SLEPTIM1"].between(1, 24)) | df["SLEPTIM1"].isna()) &
      ((df["_MENT14D"].isin([1, 2, 3])) | df["_MENT14D"].isna()) &
      ((df["INCOME3"].between(1, 11)) | df["INCOME3"].isna()) &
      ((df["EDUCA"].between(1, 6)) | df["EDUCA"].isna()) &
      ((df["_BMI5"] >= 1) | df["_BMI5"].isna())
  ]
  
  df["_BMI5"] = df["_BMI5"] / 100.0

  
  df.to_csv("cleaned_brfss_data.csv", index=False)
  
  return df
  

############ Function Call ############
data_parser()

,SLEPTIM1,_MENT14D,_BMI5,INCOME3,EDUCA
0,8.0,1.0,NaN,NaN,6.0
1,6.0,1.0,26.57,5.0,4.0
2,5.0,2.0,25.61,10.0,6.0
3,7.0,1.0,23.30,NaN,4.0
4,9.0,1.0,21.77,5.0,5.0
...,...,...,...,...,...
445127,6.0,2.0,25.63,1.0,4.0
445128,7.0,2.0,28.66,7.0,6.0
445129,5.0,3.0,17.23,NaN,4.0
445130,5.0,1.0,32.55,8.0,5.0


## Web Collection Requirement \#1


In [2]:
import pandas as pd
from bs4 import BeautifulSoup

def web_parser1():
    with open('USCODE22_LLCP_102523 4.html', 'r', encoding='windows-1252') as f:
        html = f.read()

    soup = BeautifulSoup(html, 'html.parser')

    variable_blocks = soup.find_all('td', class_='l m linecontent')

    all_variables = []

    for variable in variable_blocks:
        text = variable.get_text(separator="\n")

        variables_dict = {}
        for line in text.split('\n'):
            line = line.strip()
            if ':' in line:
                key, value = line.split(':', 1)
                key_final = ' '.join(key.split())
                variables_dict[key_final] = value.strip()

        variable_name = variables_dict.get('SAS Variable Name', 'N/A')
        variable_label = variables_dict.get('Label', 'N/A')

        table = variable.find_parent('table')
        rows = table.find_all('tr')[2:]

        values = []
        for row in rows:
            cells = row.find_all('td')
            if len(cells) >= 2:
                code = cells[0].get_text(strip=True)
                value_label = cells[1].get_text(strip=True)
                if code.lower() != 'blank' and code != '':
                    values.append((code, value_label))

        all_variables.append({
            'Variable': variable_name,
            'Label': variable_label,
            'Values': values
        })


    df_rows = []
    for var in all_variables:
        if var['Values']:
            for val_code, val_label in var['Values']:
                df_rows.append({
                    'Variable': var['Variable'],
                    'Label': var['Label'],
                    'Value Code': val_code,
                    'Value Label': val_label
                })
        else:
            # Variable without coded values
            df_rows.append({
                'Variable': var['Variable'],
                'Label': var['Label'],
                'Value Code': None,
                'Value Label': None
            })

    df = pd.DataFrame(df_rows)

    variables_of_interest = ["SLEPTIM1", "_MENT14D", "_BMI5", "INCOME3", "EDUCA"]

    df = df[df["Variable"].isin(variables_of_interest)]

    df.to_csv('parsed_codebook_data.csv', index=False)


    return df




############ Function Call ############
web_parser1()

,Variable,Label,Value Code,Value Label
237,SLEPTIM1,How Much Time Do You Sleep,1 - 24,Number of hours [1-24]
238,SLEPTIM1,How Much Time Do You Sleep,77,Don’t know/Not Sure
239,SLEPTIM1,How Much Time Do You Sleep,99,Refused
311,EDUCA,Education Level,1,Never attended school or only kindergarten
312,EDUCA,Education Level,2,Grades 1 through 8 (Elementary)
313,EDUCA,Education Level,3,Grades 9 through 11 (Some high school)
314,EDUCA,Education Level,4,Grade 12 or GED (High school graduate)
315,EDUCA,Education Level,5,College 1 year to 3 years (Some college or tec...
316,EDUCA,Education Level,6,College 4 years or more (College graduate)
317,EDUCA,Education Level,9,Refused


## Web Collection Requirement \#2

In [3]:
import requests
import pandas as pd

def web_parser2():
  url = "https://data.cdc.gov/api/views/hksd-2xuw/rows.json"
  response = requests.get(url)
  raw_data = response.json()
  
  columns = [col["fieldName"] for col in raw_data["meta"]["view"]["columns"]]
  data = raw_data["data"]
  
  df = pd.DataFrame(data, columns=columns)
  
  topics = ["Short sleep duration among adults", "Depression among adults", "Obesity among adults", "Average mentally unhealthy days among adults"]
  df = df[df["question"].isin(topics)]
  
  df = df[(df["yearstart"] == "2022") & (df["stratification1"] == "Overall")]
  
  df = df[["yearstart", "locationabbr", "locationdesc", "topic", "question", "datavalue"]]
  
  df = df.dropna(subset=["datavalue"])
  df["datavalue"] = pd.to_numeric(df["datavalue"])
  
  df.to_csv("cleaned_json_data.csv", index=False)
  
  return df


############ Function Call ############
web_parser2()

,yearstart,locationabbr,locationdesc,topic,question,datavalue
937,2022,GA,Georgia,Sleep,Short sleep duration among adults,38.8
953,2022,IL,Illinois,Sleep,Short sleep duration among adults,36.0
1077,2022,UT,Utah,"Nutrition, Physical Activity, and Weight Status",Obesity among adults,31.1
1100,2022,WY,Wyoming,"Nutrition, Physical Activity, and Weight Status",Obesity among adults,34.2
196983,2022,FL,Florida,"Nutrition, Physical Activity, and Weight Status",Obesity among adults,31.7
...,...,...,...,...,...,...
308795,2022,VT,Vermont,Mental Health,Average mentally unhealthy days among adults,5.1
308955,2022,UT,Utah,Sleep,Short sleep duration among adults,34.3
309052,2022,WY,Wyoming,Sleep,Short sleep duration among adults,32.3
309132,2022,WI,Wisconsin,Mental Health,Depression among adults,24.2


## Additional Dataset Parsing/Cleaning Functions

Write any supplemental (optional) functions here.

In [4]:
def extra_source1():
    pass

    
############ Function Call ############
extra_source1()

In [5]:
# Define further extra source functions as necessary

#Inconsistencies
For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. In the downloaded 2022 BRFSS .txt file, many variables use numeric codes like 77, 88, or 99 to indicate responses such as "Don't know,Refused, or Missing." These coded values are inconsistent with the valid numeric ranges and are not meaningful for analysis. I identified this issue using the HTML codebook and handled it by replacing these invalid codes with NaN (missing values). This will allow for standardized missing data representation in later visualization.

2. The datavalue field in the JSON dataset contains numeric values but can also sometimes contain strings, causing inconsistencies in the column’s data type. This discrepancy can lead to errors during data processing and analysis. To handle this, I converted the datavalue column to numeric using pd.to_numeric(), to make consistent numeric types across the dataset and preventing potential errors.

3. In the BRFSS dataset, the _BMI5 variable is recorded as an integer multiplied by 100 (2657 means 26.57 BMI). This representation is inconsistent with the standard BMI scale, which typically ranges from 10–30+. I addressed this inconsistency by dividing the values by 100 to accurately reflect true BMI measurements and make sure future visualizations are accurate.
